# Abstract Matrices

In [11]:
import finite_algebras as alg
import numpy as np
from abstract_matrix import AbstractMatrix

import itertools as it

import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

ex = alg.Examples(alg_dir)

                           Example Algebras
----------------------------------------------------------------------
  17 example algebras are available.
  Use "Examples[INDEX]" to retrieve a specific example,
  where INDEX is the first number on each line below:
----------------------------------------------------------------------
0: A4 -- Alternating group on 4 letters (AKA Tetrahedral group)
1: D3 -- https://en.wikipedia.org/wiki/Dihedral_group_of_order_6
2: D4 -- Dihedral group on four vertices
3: Pinter29 -- Non-abelian group, p.29, 'A Book of Abstract Algebra' by Charles C. Pinter
4: RPS -- Rock, Paper, Scissors Magma
5: S3 -- Symmetric group on 3 letters
6: S3X -- Another version of the symmetric group on 3 letters
7: V4 -- Klein-4 group
8: Z4 -- Cyclic group of order 4
9: F4 -- Field with 4 elements (from Wikipedia)
10: mag_id -- Magma with Identity
11: Example 1.4.1 -- See: Groupoids and Smarandache Groupoids by W. B. Vasantha Kandasamy
12: Ex6 -- Example 6: http://www-groups.m

In [12]:
f2 = ex[16]  # No. 16 in the list above
f4 = ex[9]  # No. 9 in the list above

In [13]:
def random_matrix_test(shape, algebra, seed=None):
    if seed:
        np.random.seed(seed)
    mat = AbstractMatrix.random(shape, algebra)
    det, inv = print_matrix_info(mat)
    #print(f"\nRandom Matrix over {algebra.name}:\n{mat}")
    return mat, det, inv

def print_matrix_info(mat):
    print("="*20)
    print(f"Matrix over {mat.algebra.name}:\n{mat}")
    det = mat.determinant()
    print(f"\nDeterminant = {det}")
    print(f"\nCofactor Matrix:\n{mat.cofactor_matrix()}")
    inv = mat.inverse()
    print(f"\nInverse:\n{inv}")
    print(f"\nMatrix * Inverse:\n{mat * mat.inverse()}")
    print(f"\nInverse * Matrix:\n{mat.inverse() * mat}\n")
    print("-"*5)
    return det, inv

In [14]:
m1, d1, i1 = random_matrix_test((3, 3), f4, seed=1)

Matrix over F4:
[['1' '1+a' '0']
 ['0' '1+a' '1']
 ['1+a' '1' '1+a']]

Determinant = 1

Cofactor Matrix:
[['1+a' '1+a' 'a']
 ['a' '1+a' '1+a']
 ['1+a' '1' '1+a']]

Inverse:
[['1+a' 'a' '1+a']
 ['1+a' '1+a' '1']
 ['a' '1+a' '1+a']]

Matrix * Inverse:
[['1' '0' '0']
 ['0' '1' '0']
 ['0' '0' '1']]

Inverse * Matrix:
[['1' '0' '0']
 ['0' '1' '0']
 ['0' '0' '1']]

-----


## Invertible 2x2 Matrices over a Finite Field

### Generate all Possible NxN Matrices over an Algebra

In [ ]:
def generate_all_NxN_matrices_over_algebra(algebra, N):
    """Generate list of all possible NxN matrices using elements from 'algebra'.
    """
    Nsqr = N * N
    combos = list(it.combinations_with_replacement(algebra.elements, Nsqr))
    perms = [list(set(it.permutations(combo))) for combo in combos]
    arrays = [AbstractMatrix(np.array((item)).reshape(N, N), algebra)
              for p in perms for item in p]
    return arrays

In [52]:
all_2x2_f2 = generate_all_NxN_matrices_over_algebra(f2, 2)
print(len(all_2x2_f2))
all_2x2_f2

16


[[['0', '0'],
  ['0', '0']],
 [['0', '0'],
  ['1', '0']],
 [['0', '1'],
  ['0', '0']],
 [['1', '0'],
  ['0', '0']],
 [['0', '0'],
  ['0', '1']],
 [['1', '0'],
  ['1', '0']],
 [['1', '0'],
  ['0', '1']],
 [['1', '1'],
  ['0', '0']],
 [['0', '0'],
  ['1', '1']],
 [['0', '1'],
  ['1', '0']],
 [['0', '1'],
  ['0', '1']],
 [['1', '0'],
  ['1', '1']],
 [['1', '1'],
  ['0', '1']],
 [['0', '1'],
  ['1', '1']],
 [['1', '1'],
  ['1', '0']],
 [['1', '1'],
  ['1', '1']]]

In [53]:
def swap_items(lst, index1, index2):
    """Swap two items in a list. This changes the input list.
    """
    lst[index1], lst[index2] = lst[index2], lst[index1]
    return lst

In [54]:
id_2x2_f2 = AbstractMatrix.identity(2, f2)
id_index = all_2x2_f2.index(id_2x2_f2)
id_index

6

In [55]:
all_2x2_f2[id_index]

[['1', '0'],
 ['0', '1']]

In [56]:
def move_identities_to_front(matrices):
    """Given a list of square matrices, all with the same shape, and over the same algebra,
    move the additive and multiplicative identities, if they're present in the list,
    to the 0 & 1 positions in the list, resp. If only one of the identities is present,
    move it to the front of the list.
    """
    
    # All the matrices must have the same square shape & be over the same algebra:
    mat0 = matrices[0]
    A = mat0.algebra
    N = mat0.shape[0]
    
    # The additive and multiplicative identities:
    id_add = AbstractMatrix.zeros((N, N), A)
    id_mul = AbstractMatrix.identity(N, A)
    
    # Booleans (True/False) as to whether the identities are present in the list
    has_add_id = id_add in matrices
    has_mul_id = id_mul in matrices

    # If the list contains the zero matrix,...
    if has_add_id:
        id_add_pos = matrices.index(id_add)

        # ...put it in the 0_th position, if it's not already there.
        if id_add_pos != 0:
            swap_items(matrices, 0, id_add_pos)

    # If the list contains the identity matrix,...
    if has_mul_id:
        id_mul_pos = matrices.index(id_mul)
        
        # ...and it has the zero matrix...
        if has_add_id:
            
            # ...put the identity matrix in position 1, if it's not already there,...
            if id_mul_pos != 1:
                swap_items(matrices, 1, id_mul_pos)
                
        # ...otherwise put the identiy in position 0, if it's not already there.
        else:
            if id_mul_pos != 0:
                swap_items(matrices, 0, id_mul_pos)
        
    return matrices

In [58]:
_ = move_identities_to_front(all_2x2_f2)
all_2x2_f2

[[['0', '0'],
  ['0', '0']],
 [['1', '0'],
  ['0', '1']],
 [['0', '1'],
  ['0', '0']],
 [['1', '0'],
  ['0', '0']],
 [['0', '0'],
  ['0', '1']],
 [['1', '0'],
  ['1', '0']],
 [['0', '0'],
  ['1', '0']],
 [['1', '1'],
  ['0', '0']],
 [['0', '0'],
  ['1', '1']],
 [['0', '1'],
  ['1', '0']],
 [['0', '1'],
  ['0', '1']],
 [['1', '0'],
  ['1', '1']],
 [['1', '1'],
  ['0', '1']],
 [['0', '1'],
  ['1', '1']],
 [['1', '1'],
  ['1', '0']],
 [['1', '1'],
  ['1', '1']]]

## Find All Matrices with Determinant '1'

In [59]:
all_2x2_f2_det1 = [m for m in allmats if m.determinant() == '1']
print(len(all_2x2_f2_det1))
all_2x2_f2_det1

6


[[['1', '0'],
  ['0', '1']],
 [['0', '1'],
  ['1', '0']],
 [['1', '0'],
  ['1', '1']],
 [['1', '1'],
  ['0', '1']],
 [['0', '1'],
  ['1', '1']],
 [['1', '1'],
  ['1', '0']]]

In [60]:
move_identities_to_front(all_2x2_f2_det1)

[[['1', '0'],
  ['0', '1']],
 [['0', '1'],
  ['1', '0']],
 [['1', '0'],
  ['1', '1']],
 [['1', '1'],
  ['0', '1']],
 [['0', '1'],
  ['1', '1']],
 [['1', '1'],
  ['1', '0']]]

## Closure of Matrices

In [67]:
def closure_of_matrices(matrices, operator):
    """Given a list of AbstractMatrix, return its closure, with respect to
    a given binary operator.
    """
    result = matrices

    # For every pair of matrices, compute their product
    # and, if new, add the product to the list
    for pair in it.product(result, result):
        # prod = pair[0] * pair[1]
        prod = operator(pair[0], pair[1])
        if prod not in result:
            result.append(prod)

    # If the operations above have enlarged the list,
    # then recursively call the function with the new,
    # expanded list.
    if len(result) > len(matrices):
        return closure_of_matrices(result)
    else:
        # Otherwise, if there's been no change to the
        # list of matrices, then return it
        return result

Matrices with determinant '1' are a closed subset of the 256 possible matrices.

In [68]:
all_2x2_f2_det1_closed = closure_of_matrices(all_2x2_f2_det1, lambda x,y: x * y)
print(len(all_2x2_f2_det1_closed))

6


In [69]:
all_2x2_f2_det1_closed

[[['1', '0'],
  ['0', '1']],
 [['0', '1'],
  ['1', '0']],
 [['1', '0'],
  ['1', '1']],
 [['1', '1'],
  ['0', '1']],
 [['0', '1'],
  ['1', '1']],
 [['1', '1'],
  ['1', '0']]]

In [71]:
all_2x2_f2_closed = closure_of_matrices(all_2x2_f2, lambda x,y: x + y)
print(len(all_2x2_f2_closed))
all_2x2_f2_closed

16


[[['0', '0'],
  ['0', '0']],
 [['1', '0'],
  ['0', '1']],
 [['0', '1'],
  ['0', '0']],
 [['1', '0'],
  ['0', '0']],
 [['0', '0'],
  ['0', '1']],
 [['1', '0'],
  ['1', '0']],
 [['0', '0'],
  ['1', '0']],
 [['1', '1'],
  ['0', '0']],
 [['0', '0'],
  ['1', '1']],
 [['0', '1'],
  ['1', '0']],
 [['0', '1'],
  ['0', '1']],
 [['1', '0'],
  ['1', '1']],
 [['1', '1'],
  ['0', '1']],
 [['0', '1'],
  ['1', '1']],
 [['1', '1'],
  ['1', '0']],
 [['1', '1'],
  ['1', '1']]]

## Create Dict of Matrices

In [45]:
def name_mapping(matrices, prefix = 'a', id_element = None, id_name = 'e'):
    n = len(matrices)
    nfill = len(str(n - 1))  # Number of zeros to left-fill integers in element names
    names = [prefix + str(i).zfill(nfill) for i in range(n)]
    if id_element:
        names[matrices.index(id_element)] = id_name
    return dict(zip(names, matrices))

In [14]:
def derive_algebra_from_matrices(matrices, element_name_prefix='a', identity_name='e'):

    m0 = matrices[0]
    A = m0.algebra
    n = m0.shape[0]
    e = AbstractMatrix.identity(n, A)

    mapping = name_mapping(matrices, element_name_prefix, e, identity_name)
    
    def get_matrix(name):
        return mapping[name]
    
    def matrix_to_tuple(matrix):
        return tuple(map(lambda x: tuple(x), matrix.array.tolist()))
    
    inv_mapping = {matrix_to_tuple(matrix): name for name, matrix in mapping.items()}
    
    def get_elem(matrix):
        return inv_mapping[matrix_to_tuple(matrix)]
    
    elems = list(mapping.keys())
    
    table = [[get_elem(get_matrix(a) * get_matrix(b)) for b in elems] for a in elems]
    
    matrix_alg = alg.make_finite_algebra(f"{A.name} Matrix Algebra",
                                         f"Algebra derived from {n}x{n} matrices over {A.name}",
                                         elems,
                                         table
                                        )
    
    return matrix_alg, get_matrix, get_elem, mapping

In [15]:
foo, xmat, xelem, xmap = derive_algebra_from_matrices(det1s, 'a', 'e')
foo

Group(
'F2 Matrix Algebra',
'Algebra derived from 2x2 matrices over F2',
['e', 'a1', 'a2', 'a3', 'a4', 'a5'],
[[0, 1, 2, 3, 4, 5], [1, 0, 3, 2, 5, 4], [2, 5, 0, 4, 3, 1], [3, 4, 1, 5, 2, 0], [4, 3, 5, 1, 0, 2], [5, 2, 4, 0, 1, 3]]
)

In [16]:
a2 = xmat('a2')
a2

[['1' '1']
 ['0' '1']]

In [17]:
xelem(a2)

'a2'

In [20]:
xmap

{'e': [['1' '0']
  ['0' '1']],
 'a1': [['0' '1']
  ['1' '0']],
 'a2': [['1' '1']
  ['0' '1']],
 'a3': [['0' '1']
  ['1' '1']],
 'a4': [['1' '0']
  ['1' '1']],
 'a5': [['1' '1']
  ['1' '0']]}

In [21]:
bar, ymat, yelem, ymap = derive_algebra_from_matrices(allmats, 'b', 'e')

In [22]:
bar.about(max_size=16)


** Monoid **
Name: F2 Matrix Algebra
Instance ID: 4578124176
Description: Algebra derived from 2x2 matrices over F2
Order: 16
Identity: e
Associative? Yes
Commutative? No
Cyclic?: No
Elements: ['b00', 'b01', 'b02', 'b03', 'b04', 'e', 'b06', 'b07', 'b08', 'b09', 'b10', 'b11', 'b12', 'b13', 'b14', 'b15']
Has Inverses? No
Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 3, 1, 7, 0, 3, 3, 1, 1, 7, 7, 3, 7],
 [0, 0, 2, 0, 4, 2, 6, 0, 4, 4, 2, 2, 6, 6, 4, 6],
 [0, 1, 0, 3, 0, 3, 0, 7, 1, 3, 1, 7, 1, 3, 7, 7],
 [0, 2, 0, 4, 0, 4, 0, 6, 2, 4, 2, 6, 2, 4, 6, 6],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 [0, 2, 2, 4, 4, 6, 6, 6, 6, 0, 0, 4, 4, 2, 2, 0],
 [0, 1, 1, 3, 3, 7, 7, 7, 7, 0, 0, 3, 3, 1, 1, 0],
 [0, 2, 1, 4, 3, 8, 7, 6, 5, 9, 10, 12, 11, 14, 13, 15],
 [0, 10, 0, 9, 0, 9, 0, 15, 10, 9, 10, 15, 10, 9, 15, 15],
 [0, 0, 10, 0, 9, 10, 15, 0, 9, 9, 10, 10, 15, 15, 9, 15],
 [0, 1, 10, 3, 9, 11, 15, 7, 14, 4, 2, 5, 13, 12, 8, 6],


In [23]:
ymat('e')

[['1' '0']
 ['0' '1']]

In [24]:
bar_subs = bar.proper_subalgebras()

In [25]:
bar_subgrps = [bs for bs in bar_subs if isinstance(bs, alg.Group)]

In [26]:
for grp in bar_subgrps:
    print("-------------------------------------")
    grp.about()

-------------------------------------

** Group **
Name: F2 Matrix Algebra_subalgebra_8
Instance ID: 4578256464
Description: Subalgebra of: Algebra derived from 2x2 matrices over F2
Order: 2
Identity: e
Commutative? Yes
Cyclic?: Yes
  Generators: ['b13']
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1     b13     b13       2
Cayley Table (showing indices):
[[0, 1], [1, 0]]
-------------------------------------

** Group **
Name: F2 Matrix Algebra_subalgebra_53
Instance ID: 4578203472
Description: Subalgebra of: Algebra derived from 2x2 matrices over F2
Order: 6
Identity: e
Commutative? No
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0       e       e       1
      1     b08     b08       2
      2     b11     b11       2
      3     b12     b14       3
      4     b13     b13       2
      5     b14     b12       3
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5],
 [1, 0, 3, 2, 5, 4],
 [2, 5, 0, 4, 3, 1],
 [3, 4, 1, 5, 2, 0],
 [4, 3

In [27]:
det1dict = name_mapping(det1s, prefix = 'a', id_element = AbstractMatrix.identity(2, f2), id_name = 'e')
det1dict

{'e': [['1' '0']
  ['0' '1']],
 'a1': [['0' '1']
  ['1' '0']],
 'a2': [['1' '1']
  ['0' '1']],
 'a3': [['0' '1']
  ['1' '1']],
 'a4': [['1' '0']
  ['1' '1']],
 'a5': [['1' '1']
  ['1' '0']]}

In [39]:
def matrix(name):
    return det1dict[name]

In [40]:
A1 = matrix('a1')
A1

KeyError: 'a1'

In [ ]:
def matrix_to_tuple(matrix):
    return tuple(map(lambda x: tuple(x), matrix.array.tolist()))

In [ ]:
matrix_to_tuple(A1)

In [ ]:
inv_mapping = {matrix_to_tuple(matrix): name for name, matrix in det1dict.items()}

In [ ]:
def elem(matrix):
    return inv_mapping[matrix_to_tuple(matrix)]

In [ ]:
elem(A1)

In [ ]:
elems = list(det1dict.keys())
print(elems)

In [ ]:
f2_table = [[elem(matrix(a) * matrix(b)) for b in elems] for a in elems]

In [ ]:
len(f2_table)

In [ ]:
f2_table

In [ ]:
f2_matrix_alg = alg.make_finite_algebra("F2 Matrix Algebra",
                                        "Algebra derived from 2x2 matrices over F2",
                                        elems,
                                        f2_table)

In [ ]:
f2_matrix_alg.about()

In [ ]:
f2_matrix_alg_subs = f2_matrix_alg.proper_subalgebras()
f2_matrix_alg_subs

In [ ]:
partitions = alg.partition_into_isomorphic_lists(f2_matrix_alg_subs)
partitions

In [ ]:
alg.about_isomorphic_partitions(f2_matrix_alg, partitions)

In [ ]:
g0 = alg.generate_symmetric_group(3)
g0.about()

In [ ]:
f2_matrix_alg.isomorphic(g0)